In [2]:
#imports
import numpy as np
import pandas as pd
import pandas_datareader as web
import matplotlib.pyplot as plt
import datetime

from scipy.stats import truncnorm
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd).rvs()

In [3]:
##################################################################################################
#####################Calculation for Market Order Price Improvement###############################
##################################################################################################
#using average statistics from a broker, Virtu Americas, these are the execution prices and times
#execution prices and times depend on the price improvement possible, and this depends on order size
def market(ordersize, buy, price):
    #print(ordersize,buy,price)
    if 0 <= ordersize <= 99.0:
        price_improvement = (.38/26.0)
    if 99.0 < ordersize <= 499:
        price_improvement = (2.70/196.0)
    if 499< ordersize <= 1999:
        price_improvement = (7.17/852)
    if 1999 < ordersize <= 4999:
        price_improvement = (10.01/2731)
    if 4999 < ordersize:
        price_improvement = (5.98/6232) 
    price_improvement = price_improvement * get_truncated_normal(0.5, np.sqrt(0.5), 0,1)
    if buy:
        execprice = price - price_improvement
    else:
        execprice = price + price_improvement     
    return round(execprice,4)

In [4]:
##################################################################################################
#####################Calculation for Total Price (per share)######################################
##################################################################################################
#check if the desired order adds or removes liquidity
#execution time will be considered instant for marketable orders that are not market orders
#execution price and time for market orders and non-marketable orders are calculated using functions
#execution time is in milliseconds
#using this syntax, a limit price input of 0 indicates a market order
def CostCalc(price,ordervalue):  
    if ordervalue >= 0:
        buy = True
        sign = 1.0
    if ordervalue < 0:
        buy = False
        sign = -1.0
    ordersize = np.abs(ordervalue)/price
    execprice = market(ordersize, buy, price)               
    if buy:
        totalprice = execprice + 0.1
    else:
        totalprice = execprice - 0.1
    if ordervalue ==0:
        totalprice = 0
        sign = 0
    return (sign*totalprice * ordersize, ordersize,ordervalue)
CostCalc_v = np.vectorize(CostCalc)

In [5]:
stocks = ['IWD', 'IWF', 'IWN', 'IWO', 'SPAB','PFOAX','WIP',  'TFI', 'JNK', 'EWX', 'ADRE', 'IEV', 'IEUS']

#date range for data
start_date = datetime.datetime(2007,3,29)
end_date = datetime.datetime(2019,3,29)

#import data for the given ticker, ^GSPC, in this case
df = web.DataReader(stocks,data_source='yahoo',start=start_date,end=end_date) 

#format and clean up the data
df = df.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis = 1)
df.columns = df.columns.droplevel('Attributes')

In [47]:
df1 = pd.read_excel('/home/terminal_1/Downloads/Data_for_projects.xlsx', 'Sheet1', header=None)
listofdates = []
for i in df1[0]:
    val = datetime.datetime.strptime(str(i), "%Y%m%d") 
    listofdates.append(val)
listofdates = pd.DatetimeIndex(listofdates)
df_ = df.loc[listofdates]
weights = df_.copy()

risk = pd.read_excel('/home/terminal_1/Downloads/DTB6.xls', header=None )
risk.columns = ["Dates", "Rates"]
risk.dropna(inplace=True)
risk.index = risk["Dates"]
risk["Rates"] = risk["Rates"] / 100.0
risk.drop(inplace = True, columns = "Dates")
riskfree = risk.loc[listofdates]

cpi = pd.read_excel('/home/terminal_1/Downloads/Monthly_Inflation_Rate.xlsx')
cpi.dropna(inplace=True)
cpi.index = cpi["Date"]
cpi.drop(inplace = True, columns = "Date")
cpirate = ((cpi.max(axis = 0) + cpi.mean(axis = 0)) /2.0)[0] 

for row in range(0,len(weights.index)):
    weights.iloc[row,:] = list((1/(len(df_.columns) - df_.isnull().sum(axis=1))))[row]

weights[df_.isnull()] = 0
weights.iloc[0] = 0
returns = df_.pct_change(-1,axis='rows')
returns["Risk_free"] = (((riskfree -(cpirate))/12.0)) - 1
df_ = df_.fillna(np.inf)

def posttransaction(rowidx, portfolioval, portfolio):
    desired_vals = weights.iloc[rowidx,:] * portfolioval
    current_vals = portfolio[:-1]
    transactions = desired_vals - current_vals
    vec = CostCalc_v(df_.iloc[rowidx,:],transactions)
    debt = np.sum(vec[2]) - np.sum(vec[0])
    transactions_matrix.iloc[rowidx] = np.append(transactions,debt)
    return np.append(desired_vals,debt)

In [48]:
listofwts = [0.019996752, 0.003534921, 0.016114844, 0.134314782, 0.14090866, 0.138093262, 0.013524113 ,0.013081445 ,0.019657952, 0.278427152, 0.181614136, 0.020080302, 0.020651677]
for i in range(1,131):
    weights.iloc[i] = listofwts

In [49]:
initial = 210000
portfolio = returns.copy()
transactions_matrix = returns.copy()
transactions_matrix.iloc[0] = 0.0
portfolio_val_after_a_month = initial
portfolio_after_a_month = np.zeros_like(np.append(weights.iloc[0,:],0))
#on each day/index calculates the portfolio val for the next period, a month later
for row in range(len(df_.index)-1,0,-1):
    portfolio_post_trans = posttransaction(row,portfolio_val_after_a_month, portfolio_after_a_month)
    if row == len(df_.index)-1:
        portfolio.iloc[row] = portfolio_post_trans
    portfolio_after_a_month = portfolio_post_trans * (1+returns.iloc[row-1])
    portfolio.iloc[row-1] = portfolio_after_a_month
    portfolio_val_after_a_month = np.sum(portfolio_after_a_month)
    if (row-1)%12 == 0:
        portfolio_val_after_a_month = portfolio_val_after_a_month - ((2000.0/12.0) + 200.0)
    if (row-1)%48 == 0:
        portfolio_val_after_a_month = portfolio_val_after_a_month - 20000
    
portfolio["Portfolio Value"] = portfolio.sum(axis=1)

https://www.nasdaqtrader.com/Trader.aspx?id=PriceListTrading2#route  
http://www.nasdaqtrader.com/Trader.aspx?id=DPUSdata  
http://www.nasdaqtrader.com/content/marketregulation/membership/NASDAQSROMembershipApplicationFinal.pdf

In [44]:
weights.to_excel('weights.xlsx')
portfolio.to_excel('portfolio.xlsx')
transactions_matrix.to_excel('transactions.xlsx')